In [6]:
pip install transformers datasets evaluate

  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.59.0
    Uninstalling tqdm-4.59.0:
      Successfully uninstalled tqdm-4.59.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 0.9.0
    Uninstalling fsspec-0.9.0:
      Successfully uninstalled fsspec-0.9.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.4
    Uninstalling dill-0.3.4:
      Successfully uninstalled dill-0.3.4
Note: you may need to restart the kernel to use updated packages.


In [1]:
from huggingface_hub import notebook_login

In [2]:
notebook_login()

In [3]:
from datasets import load_dataset

In [4]:
sandra = load_dataset("data")

Found cached dataset json (C:/Users/ariel/.cache/huggingface/datasets/json/data-99bffce22e1cf858/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
data = sandra['train'].train_test_split(test_size=0.15)

In [6]:
from transformers import AutoTokenizer

In [7]:
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

In [8]:
from functools import partial

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

def tokenize_function(tokenizer, examples):
    return tokenizer(examples["message"])

partial_tokenize_function = partial(tokenize_function, tokenizer)

tokenized_data = data.map(
    partial_tokenize_function,
    batched = True,
    num_proc = 4
)

Map (num_proc=4):   0%|          | 0/3533 [00:00<?, ? examples/s]

TypeError: 'GPT2Tokenizer' object is not callable

In [ ]:
tokenized_data

# Sandra

In [9]:
pip list

Package                            Version
---------------------------------- -------------------
-ensorflow-gpu                     2.6.0
absl-py                            0.13.0
aiohttp                            3.8.4
aiosignal                          1.3.1
alabaster                          0.7.12
anaconda-client                    1.7.2
anaconda-navigator                 2.0.3
anaconda-project                   0.9.1
anyio                              2.2.0
appdirs                            1.4.4
argh                               0.26.2
argon2-cffi                        20.1.0
asn1crypto                         1.4.0
astor                              0.8.1
astroid                            2.5
astropy                            4.2.1
astunparse                         1.6.3
async-generator                    1.10
async-timeout                      4.0.2
atomicwrites                       1.4.0
attrs                              20.3.0
autopep8                           1.5.